In [79]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 
from os import walk
import json 

In [80]:
def read_json(file_name):
    with open(file_name) as f:
      data = json.load(f)
    return data

In [44]:
b =  [1,   2,   3,  4,   5,    6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
bstress = [22, 10, 19, 17, 23, 14, 9, 18, 23, 10, 8, 13, 12, 20, 21, 13, 0, 28, 13, 7, 18, 13, 15]
g =  [1,    2,   3,  4,  5,  6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17,]
gstress =  [11, 13, 25, 7, 28, 14, 8, 13, 10, 12, 11, 16, 20, 25, 16, 31, 9]

In [55]:
df = pd.read_csv('../table.csv')

In [56]:
df.columns

Index(['Exp_ID', 'exp_time', 'trais no.', 'note', 'signal_condition', 'gender',
       'age', 'race', 'handiness', 'sleep condition', 'Q11', 'Q12', 'Q13',
       'Q13_r', 'Q14', 'ID', 'ID_boy', 'ID_girl', 'Raw_within_RF',
       'Raw_within_logistic', 'within_classification',
       'within_classification_logistic', 'Raw_within_netVSneg_RF',
       'Raw_within_netVSneg_logistic', 'Within_netVSneg_RF',
       'Within_netVSneg_logistic', 'within_Feature_asymmtry_RF',
       'within_Feature_Less_RF', 'within_Feature_Less_Logistic', 'stress',
       'stressGroup', '42set_ID', 'between_classification_all',
       'Between_Feature_asymmtry', 'Between_Feature_Less',
       'raw_betweenNeutral2Negative', 'raw_StressGroupbetweenNeutral2Negative',
       'raw_between_stressGroup', 'between_stressGroup',
       'Between_Stress2groups_RF', 'Between_Stress2groups_log',
       'Between_GenderGroup', 'Between_RandomGroup',
       'between_stressGroup_netural_vs_negative', '39set_ID',
       'between_

In [277]:
folder = '../results/0307/'

In [278]:
a, b, filenames = next(walk(folder))

In [279]:
print(filenames)

['leave_one_order_33_stress_l100.json', 'leave_one_order_33_stress_h0.1.json', 'leave_one_order_42_h1.json', 'within_33logistic1000.json', 'within_42_logistic100.json', 'leave_one_order_42_m50.json', 'within_42logistic_noscaler50.json', 'leaveone_33_logistic50.json', 'within_33logistic100.json', 'leave_one_order_36_h1000.json', 'leave_one_order_33_stress_m1000.json', 'leave_one_order_33_stress_l1000.json', 'leaveone_33_logistic0.1.json', '.DS_Store', 'within_42_logistic50.json', 'leave_one_order_36_l50.json', 'leave_one_order_33_stress_m100.json', 'leave_one_order_33_stress_m50.json', 'leave_one_order_42_l1.json', 'leave_one_order_36_h50.json', 'leaveone_33_logistic1000.json', 'within_39_logistic50.json', 'within_39_logistic100.json', 'leave_one_order_42_l100.json', 'leave_one_order_42_h0.1.json', 'leave_one_order_36_l0.1.json', 'leave_one_order_36_h100.json', 'leave_one_order_36_m0.1.json', 'leave_one_order_42_m1.json', 'leave_one_order_42_m100.json', 'leave_one_order_42_h1000.json', 

In [287]:
arr = []
for i in filenames:
    if '.json' in i:
        val = cal_avg(read_json(folder+i)['avg_acc'])
        arr.append([val,i])

leave_one_order_33_stress_l100.json


In [283]:
def cal_avg(arr):
    print(sum(arr)/len(arr))
cal_avg(val1['avg_acc'])
cal_avg(val2['avg_acc'])

0.712962962962963
0.7160493827160495


In [242]:
bad = read_json(folder+filenames[1])['avg_acc']
sum(bad)/len(bad)

0.7584722222222223

In [259]:
num = 0
final = [0 for _ in range(40)]
arr = []
for num in range(100):
    temp = []
    for group in range(3):
        accs = read_json(folder+f"trailNum{num}_groupNum{group}.json")['avg_acc']
        temp.append(sum(accs)/len(accs))
    arr.append(np.array(temp))
arr = np.array(arr)
np.savetxt("temp.csv", arr, delimiter=",")
# df = pd.DataFrame([])
# df['14_group'] = arr[]
# df.to_csv("temp.csv")

In [170]:
for i in range(len(final)):
    final[i] = final[i]/100
print(final)

[0.7532222222222219, 0.7348333333333337, 0.7050000000000003, 0.7224166666666667, 0.7361666666666664, 0.7274444444444446, 0.7187500000000001, 0.7183055555555553, 0.7177500000000002, 0.7241111111111114, 0.7194722222222225, 0.7198333333333337, 0.7057500000000003, 0.7090833333333335, 0.7250277777777776, 0.7173055555555556, 0.7523333333333341, 0.727138888888889, 0.7269999999999999, 0.6883888888888887, 0.7143888888888889, 0.7379722222222221, 0.6928055555555557, 0.728388888888889, 0.7242222222222223, 0.7320000000000002, 0.7216111111111111, 0.7196666666666665, 0.7150833333333331, 0.7116388888888889, 0.7205277777777781, 0.7179166666666665, 0.7065277777777779, 0.705277777777778, 0.7178611111111111, 0.729527777777778, 0.7205555555555557, 0.7174722222222223, 0.7235000000000003, 0.7162222222222225]


In [209]:
df = pd.read_csv('../table.csv')

In [202]:
vals = df['Q13_r'].values 
net =[]
neg = []
print(vals)
print(type(vals[0]))
for i in range(len(vals)):
    temp = str(vals[i])
    if 'Negative' == temp:
        neg.append(i)
    elif 'Neutral' == temp:
        net.append(i)
print(net,neg)

['Negative' 'Equal' 'Neutral' 'Neutral' 'Negative' 'Neutral' 'Negative'
 'Negative' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Equal'
 'Negative' 'Negative' 'Neutral' 'Equal' 'Neutral' 'Negative' 'Negative'
 'Negative' 'Neutral' 'Neutral' 'Neutral' 'Negative' 'Negative' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Negative' 'Neutral' 'Neutral' 'Negative'
 'Equal' 'Neutral' 'Neutral' 'Neutral' 'Neutral' nan 'Neutral' nan nan nan
 nan nan nan nan nan nan nan]
<class 'str'>
[2, 3, 5, 8, 9, 10, 11, 12, 16, 18, 22, 23, 24, 27, 28, 29, 30, 32, 33, 36, 37, 38, 39, 41] [0, 4, 6, 7, 14, 15, 19, 20, 21, 25, 26, 31, 34]


In [172]:
for _ in range(20):
    final.append(-1)
df['Between_RandomGroup'] = final[:52]
df.to_csv('../table.csv')

In [138]:
overall.sort(key=lambda x:x[0])
good.sort(key=lambda x:x[0])

In [151]:
print(overall)

[0.8888888888888888, 0.8333333333333334, 0.8611111111111112, 0.75, 0.8333333333333334, 0.8888888888888888, 0.8611111111111112, 0.6666666666666666, 0.6388888888888888, 0.75, 0.7666666666666667, 0.5666666666666667, 0.8888888888888888, 0.7333333333333333, 0.5, 0.7222222222222222, 0.8055555555555556, 0.6388888888888888, 0.5833333333333334, 0.6944444444444444, 0.75, 0.5833333333333334, 0.6666666666666666, 0.6944444444444444, 0.8333333333333334, 0.7222222222222222, 0.8611111111111112, 0.5, 0.6944444444444444, 0.6111111111111112, 0.8888888888888888, 0.8055555555555556, 0.7777777777777778, 0.6944444444444444, 0.5833333333333334, 0.8611111111111112, 0.7777777777777778, 0.75, 0.8333333333333334, 0.8, 0.6333333333333333, 0.5666666666666667]


In [140]:
for i in range(len(overall)):
    overall[i] = overall[i][1]

In [235]:
# ['bad.json', 'good.json']
bad = read_json(folder+filenames[0])['avg_acc']
good =  read_json(folder+filenames[1])['avg_acc']
id_good = [2, 3, 5, 8, 9, 10, 11, 12, 16, 18, 22, 23, 24, 27, 28, 29, 30, 32, 33, 36, 37, 38, 39, 41]  
id_bad = [0, 4, 6, 7, 14, 15, 19, 20, 21, 25, 26, 31, 34]

In [236]:
print(sum(bad)/len(bad))
print(sum(good)/len(good))

0.7329059829059829
0.7502314814814817


In [229]:
for i in range(len(bad)):
    bad[i] = [id_bad[i],bad[i]]
for i in range(len(good)):
    good[i] = [id_good[i],good[i]]
bad.extend(good)

In [230]:
bad.sort()
print(bad)
for i in range(len(bad)):
    bad[i] = bad[i][1]

[[0, 0.9166666666666666], [2, 0.8055555555555556], [3, 0.6666666666666666], [4, 0.9166666666666666], [5, 0.8333333333333334], [6, 0.8333333333333334], [7, 0.6111111111111112], [8, 0.7222222222222222], [9, 0.7222222222222222], [10, 0.7666666666666667], [11, 0.7666666666666667], [12, 0.8888888888888888], [14, 0.5555555555555556], [15, 0.5555555555555556], [16, 0.8055555555555556], [18, 0.6666666666666666], [19, 0.6388888888888888], [20, 0.8055555555555556], [21, 0.7222222222222222], [22, 0.6388888888888888], [23, 0.6944444444444444], [24, 0.8333333333333334], [25, 0.6388888888888888], [26, 0.8888888888888888], [27, 0.5833333333333334], [28, 0.6666666666666666], [29, 0.6944444444444444], [30, 0.9444444444444444], [31, 0.8055555555555556], [32, 0.6388888888888888], [33, 0.6666666666666666], [34, 0.6388888888888888], [36, 0.7777777777777778], [37, 0.7777777777777778], [38, 0.8055555555555556], [39, 0.8333333333333334], [41, 0.8055555555555556]]


[0.9166666666666666,
 0.8055555555555556,
 0.6666666666666666,
 0.9166666666666666,
 0.8333333333333334,
 0.8333333333333334,
 0.6111111111111112,
 0.7222222222222222,
 0.7222222222222222,
 0.7666666666666667,
 0.7666666666666667,
 0.8888888888888888,
 0.5555555555555556,
 0.5555555555555556,
 0.8055555555555556,
 0.6666666666666666,
 0.6388888888888888,
 0.8055555555555556,
 0.7222222222222222,
 0.6388888888888888,
 0.6944444444444444,
 0.8333333333333334,
 0.6388888888888888,
 0.8888888888888888,
 0.5833333333333334,
 0.6666666666666666,
 0.6944444444444444,
 0.9444444444444444,
 0.8055555555555556,
 0.6388888888888888,
 0.6666666666666666,
 0.6388888888888888,
 0.7777777777777778,
 0.7777777777777778,
 0.8055555555555556,
 0.8333333333333334,
 0.8055555555555556]

In [234]:
for _ in range(20):
    bad.append(-1)
df['order_difference_q13_rf'] = bad[:52]
df.to_csv('../table.csv')